## Trip data

In [3]:
!pip install holidays    # to mark holiday flag

In [4]:
import pandas as pd
import glob
import holidays

In [5]:
parquet_files = glob.glob('yellow_tripdata_2425/*.parquet')
df = pd.concat([pd.read_parquet(file) for file in parquet_files], ignore_index=True)
print(df.head())

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2024-04-01 00:02:40   2024-04-01 00:30:42              0.0   
1         2  2024-04-01 00:41:12   2024-04-01 00:55:29              1.0   
2         2  2024-04-01 00:48:42   2024-04-01 01:05:30              1.0   
3         2  2024-04-01 00:56:02   2024-04-01 01:05:09              1.0   
4         1  2024-04-01 00:08:32   2024-04-01 00:10:24              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           5.20         1.0                  N           161             7   
1           5.60         1.0                  N           264           264   
2           3.55         1.0                  N           186           236   
3           1.06         1.0                  N           137           164   
4           0.70         1.0                  N           236           263   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [6]:
print(df.head().dtypes)

VendorID                          int32
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int32
DOLocationID                      int32
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Airport_fee                     float64
dtype: object


In [7]:
df.isna().any()

VendorID                 False
tpep_pickup_datetime     False
tpep_dropoff_datetime    False
passenger_count           True
trip_distance            False
RatecodeID                True
store_and_fwd_flag        True
PULocationID             False
DOLocationID             False
payment_type             False
fare_amount              False
extra                    False
mta_tax                  False
tip_amount               False
tolls_amount             False
improvement_surcharge    False
total_amount             False
congestion_surcharge      True
Airport_fee               True
dtype: bool

passenger_count: A driver-entered value. Some drivers might not enter how many passengers are in the vehicle, and some drivers have entered 0. In addition, it's irrelavant to prediction of taxi demand in NYC.<br>
RatecodeID: Stands for the reason of the rate charged. Also irrelavant to prediction of taxi demand in NYC.<br>
store_and_fwd_flag: A flag of whether the trip is previously stored before sent to the vendor. Also irrelavant to prediction of taxi demand in NYC.<br>
congestion_surcharge: For NYS congestion surcharge. Also irrelavant to prediction of taxi demand in NYC.<br>
Airport_fee: Somewhat relevant to the taxi demand in the airports of NYC. However, it's a value missing completely at random, so we can drop it.<br>

In [9]:
# As stated above, all columns with null values can be dropped.
df_cleaned = df.dropna(axis=1, how='any')

In [10]:
# check if there is any left columns with null values.
df_cleaned.isna().any()

VendorID                 False
tpep_pickup_datetime     False
tpep_dropoff_datetime    False
trip_distance            False
PULocationID             False
DOLocationID             False
payment_type             False
fare_amount              False
extra                    False
mta_tax                  False
tip_amount               False
tolls_amount             False
improvement_surcharge    False
total_amount             False
dtype: bool

In [11]:
# Then, we can drop all columns mentioning charges, it includes payment_type, 
# fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge
df_cleaned = df_cleaned.drop(["payment_type", "fare_amount", "extra", "mta_tax", "tip_amount", 
                              "tolls_amount", "improvement_surcharge"], axis=1)

In [12]:
# Before dropping all charges, we need to assess the demand of taxi by total amount over trip distance
df_cleaned['dist_avg_amt'] = df_cleaned['total_amount'] / df_cleaned['trip_distance']

In [13]:
# check if there is any left columns with null values.
df_cleaned.isna().any()

VendorID                 False
tpep_pickup_datetime     False
tpep_dropoff_datetime    False
trip_distance            False
PULocationID             False
DOLocationID             False
total_amount             False
dist_avg_amt              True
dtype: bool

In [14]:
# drop location ID 264 and 265 due to unknown zone name
df_cleaned = df_cleaned[~(df_cleaned['PULocationID'].isin([264, 265]) & df_cleaned['DOLocationID'].isin([264, 265]))]

In [15]:
df_cleaned[df_cleaned.isnull().any(axis=1)]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,total_amount,dist_avg_amt
17642,1,2024-04-01 10:15:51,2024-04-01 10:17:14,0.0,140,140,0.0,NaN
25619,1,2024-04-01 11:05:35,2024-04-01 11:05:35,0.0,13,264,0.0,NaN
26008,1,2024-04-01 12:14:21,2024-04-01 12:15:38,0.0,207,207,0.0,NaN
37200,1,2024-04-01 14:36:56,2024-04-01 14:37:28,0.0,75,75,0.0,NaN
43121,1,2024-04-01 15:11:24,2024-04-01 15:15:37,0.0,211,231,0.0,NaN
...,...,...,...,...,...,...,...,...
44583719,1,2024-08-23 19:51:59,2024-08-23 19:52:15,0.0,43,43,0.0,NaN
44593873,1,2024-08-25 02:23:56,2024-08-25 02:30:42,0.0,255,255,0.0,NaN
44607497,1,2024-08-27 09:41:59,2024-08-27 09:51:30,0.0,90,90,0.0,NaN
44611538,1,2024-08-27 22:12:06,2024-08-27 22:12:16,0.0,93,93,0.0,NaN


We've noticed that there are some null values of averaged amount over distance, it's because either the total amount or trip distance is 0. The reason for those zeros may be that those trips have been cancelled in the past. Therefore, we still need to remove those rows from the original dataframe.

In [17]:
# As stated above, those rows with null values can be dropped.
df_cleaned = df_cleaned.dropna(axis=0, how='any')

In [18]:
# check if there is any left columns with null values.
df_cleaned.isna().any()

VendorID                 False
tpep_pickup_datetime     False
tpep_dropoff_datetime    False
trip_distance            False
PULocationID             False
DOLocationID             False
total_amount             False
dist_avg_amt             False
dtype: bool

In [19]:
# Then we drop total amount
df_cleaned = df_cleaned.drop(["total_amount"], axis=1)

In [20]:
# Now we sort the records by ascending order of pickup datetime
df_cleaned = df_cleaned.sort_values(by="tpep_pickup_datetime", ascending=True)

In [21]:
# check the first five lines of the dataframe
df_cleaned.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,dist_avg_amt
4596594,2,2002-12-31 16:46:07,2002-12-31 17:24:07,7.77,138,263,8.066924
296209,2,2002-12-31 22:10:04,2002-12-31 22:14:22,7.55,170,231,6.055629
19000195,2,2002-12-31 22:17:10,2002-12-31 22:42:24,1.40,50,162,12.857143
41059314,2,2002-12-31 22:17:43,2002-12-31 22:23:55,0.67,237,140,17.761194
611171,2,2002-12-31 22:19:28,2003-01-01 18:58:09,0.55,246,68,20.363636


The reason why there are some records as old as 2002 is that some of the records were just reported to NYC government. It seems weird so we delete them from the original dataset.

In [23]:
start = pd.to_datetime("2023-12-31")
df_cleaned = df_cleaned[df_cleaned["tpep_pickup_datetime"] >= start]

In [24]:
# Finally, we put on holiday flag

# U.S. holidays for relevant years
us_holidays = holidays.US(years=df_cleaned["tpep_pickup_datetime"].dt.year.unique())

# Convert to datetime64[ns] for safe comparison
holiday_dates = pd.to_datetime(list(us_holidays.keys()))

# Mark holiday and weekend flag
df_cleaned["is_holiday_or_weekend"] = df_cleaned["tpep_pickup_datetime"].isin(holiday_dates) | df_cleaned["tpep_pickup_datetime"].dt.weekday >= 5

In [25]:
# print out the first five rows of vendors
df_cleaned.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,dist_avg_amt,is_holiday_or_weekend
35055139,2,2023-12-31 23:39:17,2023-12-31 23:42:00,0.47,90,68,21.489362,False
35055523,2,2023-12-31 23:41:02,2023-12-31 23:48:03,0.40,246,246,30.500000,False
35057946,2,2023-12-31 23:47:28,2023-12-31 23:57:07,1.44,68,137,13.083333,False
35057755,2,2023-12-31 23:49:12,2024-01-01 00:04:32,3.14,234,237,9.108280,False
35063398,2,2023-12-31 23:54:27,2024-01-01 00:13:12,7.70,229,244,5.937662,False


In [26]:
# Export the cleaned dataframe to a CSV file
df_cleaned.to_csv('cleaned_taxi_data.csv', index=False)

## Weather data

In [28]:
# read the original data
dfw = pd.read_csv('NYC_weather_from_1979.csv')

In [29]:
# check the data types for each value
print(dfw.dtypes)

dt                       int64
dt_iso                  object
timezone                 int64
city_name               object
lat                    float64
lon                    float64
temp                   float64
visibility             float64
dew_point              float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure                 int64
sea_level              float64
grnd_level             float64
humidity                 int64
wind_speed             float64
wind_deg                 int64
wind_gust              float64
rain_1h                float64
rain_3h                float64
snow_1h                float64
snow_3h                float64
clouds_all               int64
weather_id               int64
weather_main            object
weather_description     object
weather_icon            object
dtype: object


In [30]:
# check where the null values appear
dfw.isna().any()

dt                     False
dt_iso                 False
timezone               False
city_name              False
lat                    False
lon                    False
temp                   False
visibility              True
dew_point              False
feels_like             False
temp_min               False
temp_max               False
pressure               False
sea_level               True
grnd_level              True
humidity               False
wind_speed             False
wind_deg               False
wind_gust               True
rain_1h                 True
rain_3h                 True
snow_1h                 True
snow_3h                 True
clouds_all             False
weather_id             False
weather_main           False
weather_description    False
weather_icon           False
dtype: bool

In [31]:
# convert the timestamp column from object to datetime
dfw["dt_iso"] = pd.to_datetime(dfw["dt_iso"].str.replace(" UTC", "", regex=False)).dt.tz_convert(None)

In [32]:
print(dfw['dt_iso'].dtypes)

datetime64[ns]


In [33]:
# we only need to keep several columns
columns_to_keep = ["dt_iso","temp","feels_like","temp_min","temp_max","wind_speed","rain_1h","weather_id","weather_main","weather_description"]
dfw = dfw[[col for col in columns_to_keep if col in dfw.columns]]

In [34]:
# check where the null values appear
dfw.isna().any()

dt_iso                 False
temp                   False
feels_like             False
temp_min               False
temp_max               False
wind_speed             False
rain_1h                 True
weather_id             False
weather_main           False
weather_description    False
dtype: bool

In [35]:
# since all null values appear in the column "rain_1h" when the weather is not rainy, so we can fill all of them with zero.
dfw.fillna(0.0, inplace=True)

All the temperatures are in Celcius for convenience of measurement.

In [37]:
# print the first five lines of weather data
dfw.head()

,dt_iso,temp,feels_like,temp_min,temp_max,wind_speed,rain_1h,weather_id,weather_main,weather_description
0,1979-01-01 00:00:00,6.39,2.40,5.85,7.19,6.70,0.0,741,Fog,fog
1,1979-01-01 00:00:00,6.39,2.40,5.85,7.19,6.70,0.0,310,Drizzle,light intensity drizzle rain
2,1979-01-01 00:00:00,6.39,2.40,5.85,7.19,6.70,0.0,500,Rain,light rain
3,1979-01-01 01:00:00,6.15,3.90,5.67,6.94,2.98,0.0,804,Clouds,overcast clouds
4,1979-01-01 02:00:00,6.25,3.93,5.68,7.20,3.12,0.0,804,Clouds,overcast clouds


In [38]:
# Here, if we only need the year of 2024 ~ 2025, we can select those
# However, consider that the trip data starts as old as 2002-12-31, we also need to select weather data at that time
dfw24 = dfw[
    (dfw["dt_iso"] >= "2002-12-31") & 
    (dfw["dt_iso"] < "2026-01-01")
]

In [39]:
# check the first five lines
dfw24.head()

,dt_iso,temp,feels_like,temp_min,temp_max,wind_speed,rain_1h,weather_id,weather_main,weather_description
225505,2002-12-31 00:00:00,3.00,-0.95,2.37,3.65,4.6,0.00,803,Clouds,broken clouds
225506,2002-12-31 01:00:00,3.06,-0.26,2.28,4.18,3.6,0.00,804,Clouds,overcast clouds
225507,2002-12-31 02:00:00,3.12,-1.12,2.46,4.11,5.2,0.81,500,Rain,light rain
225508,2002-12-31 03:00:00,3.82,0.66,3.09,4.69,3.6,0.69,500,Rain,light rain
225509,2002-12-31 04:00:00,3.77,0.95,3.29,4.32,3.1,0.30,500,Rain,light rain


In [40]:
# show the data types
dfw24.info()

<class 'pandas.core.frame.DataFrame'>
Index: 209417 entries, 225505 to 434921
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   dt_iso               209417 non-null  datetime64[ns]
 1   temp                 209417 non-null  float64       
 2   feels_like           209417 non-null  float64       
 3   temp_min             209417 non-null  float64       
 4   temp_max             209417 non-null  float64       
 5   wind_speed           209417 non-null  float64       
 6   rain_1h              209417 non-null  float64       
 7   weather_id           209417 non-null  int64         
 8   weather_main         209417 non-null  object        
 9   weather_description  209417 non-null  object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 17.6+ MB


In [41]:
# Export the cleaned dataframe to a CSV file
dfw24.to_csv('weather_data.csv', index=False)